Here, we'll check and build equivalence between tf and keras using simple examples.
Let's first build a conv2d block in tf (and a conv2d block in keras) and compare between them.

In [2]:
import os
import tensorflow as tf
import numpy as np

# load weights from https://dl.dropboxusercontent.com/u/50333326/vgg16.npy
path = '/home/ariel/DL/tensorflow/tutorials/'
vgg16_npy_path = os.path.join(path, 'vgg16.npy')            
data_dict = np.load(vgg16_npy_path, encoding='latin1').item()


In [3]:
# print keys in dictionary - 
# here it prints all the layers names in the VGG net
name = 'conv2_1'
print 'keys in layers = ', data_dict.keys(), '\n'
print 'weights shape in conv2_1', data_dict[name][0].shape, '\n'
print 'bias shape in conv2_1', data_dict[name][1].shape, '\n'

keys in layers =  ['conv5_1', 'fc6', 'conv5_3', 'fc7', 'fc8', 'conv5_2', 'conv4_1', 'conv4_2', 'conv4_3', 'conv3_3', 'conv3_2', 'conv3_1', 'conv1_1', 'conv1_2', 'conv2_2', 'conv2_1'] 

weights shape in conv2_1 (3, 3, 64, 128) 

bias shape in conv2_1 (128,) 



Now let's build a convolution block in tf. A convoution block is built out of:
1. a 2d convolution
2. an added bias
3. a relu module
All the code bellow is taken from:
https://github.com/MarvinTeichmann/tensorflow-fcn/blob/master/fcn16_vgg.py

In the code bellow the function tf.get_variable() is used to get/create a variable instead 
of a direct call to tf.Variable(). It uses an initalizer - 3 kind:
1. tf.constant_initializer(value) initializes everything to the provided value
2. tf.random_uniform_initializer(a, b) initializes uniformly from [a, b]
3. tf.random_normal_initializer(mean, stddev) initializes from the normal distribution with the given mean and standard deviation


In [4]:
def _bias_reshape(bweight, num_orig, num_new):
    """ Build bias weights for filter produces with `_summary_reshape`
    """
    n_averaged_elements = num_orig//num_new
    avg_bweight = np.zeros(num_new)
    for i in range(0, num_orig, n_averaged_elements):
        start_idx = i
        end_idx = start_idx + n_averaged_elements
        avg_idx = start_idx//n_averaged_elements
        if avg_idx == num_new:
            break
        avg_bweight[avg_idx] = np.mean(bweight[start_idx:end_idx])
    return avg_bweight

# need to understand this! Only for fc8, the bias is recalculated!
def get_bias(name, num_classes=None):
    bias_wights = data_dict[name][1]
    shape = data_dict[name][1].shape
    if name == 'fc8':
        bias_wights = _bias_reshape(bias_wights, shape[0], num_classes)
        shape = [num_classes]
    init = tf.constant_initializer(value=bias_wights, dtype=tf.float32)
    return tf.get_variable(name="biases", initializer=init, shape=shape)

def get_conv_filter(name):
    with tf.variable_scope("C"):
        init = tf.constant_initializer(value=data_dict[name][0], dtype=tf.float32)
        shape = data_dict[name][0].shape
        print('Layer name: %s' % name)
        print('Layer shape: %s' % str(shape))
        var = tf.get_variable(name="filter", initializer=init, shape=shape)
#     if not tf.get_variable_scope().reuse:
#         weight_decay = tf.multiply(tf.nn.l2_loss(var), self.wd,
#                                    name='weight_loss')
#         tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES,
#                              weight_decay)
    return var

def get_weights(name):
    init = tf.constant_initializer(value=data_dict[name][0], dtype=tf.float32)
    weights = tf.get_variable(name="weights", initializer=init, shape=data_dict[name][0].shape)
    return weights

def get_biases(name):
    init = tf.constant_initializer(value=data_dict[name][1], dtype=tf.float32)
    biases = tf.get_variable(name="biases", initializer=init, shape=data_dict[name][1].shape)
    return biases

def _conv_layer(bottom, name):
    with tf.variable_scope(name):# as scope:
        # get filter weights
        weights = get_weights(name)
       
        # get filter biases
        biases = get_biases(name)

        # convolve: weights * bottom
        conv = tf.nn.conv2d(bottom, weights, [1, 1, 1, 1], padding='SAME')

        # add biases: weights * bottom + biases
        conv_biases = tf.nn.bias_add(conv, biases)

        # relu: relu(weights * bottom + biases)
        conv_biases_relu = tf.nn.relu(conv_biases)

        return conv_biases_relu


Now let's run the _conv_layer on a simple image

In [8]:
# this command clean all nodes in the graph + variables
tf.reset_default_graph()

# load an image
IMAGE_SIZE = 224
img_shape = (1, IMAGE_SIZE, IMAGE_SIZE, 3) # tf format (N,H,W,C)
img = np.linspace(-0.1, 0.5, num=np.prod(img_shape)).reshape(img_shape)
tf_img = tf.Variable(img, name="img", dtype=tf.float32)

_conv1_1 = _conv_layer(tf_img, 'conv1_1')
# _conv1_2 = _conv_layer(_conv1_1, 'conv1_2')

sess = tf.Session()
sess.run(tf.initialize_all_variables())
sess.run(_conv1_1) 

array([[[[ 1.05118346,  0.07285573,  0.07526198, ...,  1.41887295,
           0.50928986,  0.50768656],
         [ 0.97429979,  0.07416046,  0.00820402, ...,  1.48566544,
           0.30226624,  0.37049884],
         [ 0.97427028,  0.07416278,  0.00821107, ...,  1.48564315,
           0.30229396,  0.37051302],
         ..., 
         [ 0.96782184,  0.07467214,  0.00975351, ...,  1.48075283,
           0.30836394,  0.37361217],
         [ 0.96779239,  0.07467446,  0.00976057, ...,  1.48073053,
           0.30839169,  0.37362629],
         [ 0.72999299,  0.08021346,  0.        , ...,  1.40367281,
           0.17518124,  0.21808174]],

        [[ 0.94112337,  0.06718749,  0.10576713, ...,  1.48901117,
           0.72521597,  0.62522233],
         [ 0.72031772,  0.07191091,  0.02346974, ...,  1.58839536,
           0.53925383,  0.48371553],
         [ 0.7203179 ,  0.07191356,  0.02347515, ...,  1.58836007,
           0.53925395,  0.48371652],
         ..., 
         [ 0.72034842,  0.072495

Now, let's do the same with keras.

In [ ]:
# build a simple model

# add a conv2d layer to this model with initial weights and biases

# run on a simple image